In [1]:
# core libraries
import pickle
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings("ignore")
from datetime import datetime


from queries import final_features
from score_funcs import Score
sc = Score()


/Users/shashankgupta/Documents/code/git_project/re_decisioning
/Users/shashankgupta/Documents/code/git_project/re_decisioning/Code


In [2]:
os.chdir('../')
cwd = os.getcwd()
print(cwd)
 
os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

from config.config import SQLQuery
querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/re_decisioning
/Users/shashankgupta/Documents/code/git_project/re_decisioning/Code


#### Data pull for scoring 

In [3]:
querySno = SQLQuery('snowflake')
df = querySno(final_features)

In [4]:
df.head()

,business_id,due_date,median_running_balance_2nd_m,median_amount_debited_2m,distinct_debit_txns_1m,ratio_debit_credit_2m,ratio_ach_credit_freq_total_credit_1m,ratio_amt_drawn_mom,ratio_ach_debit_credit_freq_1m,ratio_amt_cashin_mom,ratio_revenue_1m_2m,distinct_cashins_1m_prev
0,e48c2153-6bec-4711-92b6-04f1fcbf8b62,2022-12-23,114.89,2.00,42,1.305130,0.736842,0.000000,0.000000,0.000000,0.177905,0.0
1,4a95d0ac-791c-4c26-ba04-27529cf78788,2022-12-24,57.47,0.00,14,0.621232,0.062500,0.089158,0.000000,1.979837,0.043739,7.5
2,4a95d0ac-791c-4c26-ba04-27529cf78788,2022-12-29,77.47,0.00,9,0.621232,0.043478,0.089158,0.000000,1.979837,0.043739,7.5
3,c2feb1d5-f5a9-4fde-b74a-09775639b486,2022-12-31,131.92,27.94,39,2.329286,0.692308,0.000000,0.600000,0.000000,2.871674,0.0
4,0171a1ff-d47d-4029-b4b2-c1d2b579b857,2022-12-31,4905.99,33.54,42,0.876278,0.625000,0.000000,1.166667,393.330000,0.568249,1.0


In [5]:
df.shape

(5725, 12)

In [6]:
data_params = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/re_decisioning/models/data_params_v2.pkl')
model = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/re_decisioning/models/redec_3.pkl')
transformer = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/re_decisioning/models/scale_train_object.pkl')

In [7]:
data_params

,feature,median,lower_limit,upper_limit
45,distinct_debit_txns_1m,40.000000,1.0000,815.000000
52,median_amount_debited_2m,11.590000,0.0000,263.691700
56,ratio_debit_credit_2m,1.021660,0.4622,3.647596
70,ratio_ach_credit_freq_total_credit_1m,0.522774,0.0000,0.963411
80,ratio_ach_debit_credit_freq_1m,0.750000,0.0000,11.000000
93,median_running_balance_2nd_m,971.005000,-240.9432,28671.967000
99,ratio_revenue_1m_2m,0.998223,0.0000,772.812175
107,distinct_cashins_1m_prev,1.000000,0.0000,12.000000
109,ratio_amt_drawn_mom,0.000000,0.0000,8115.000000
110,ratio_amt_cashin_mom,1.161163,0.0000,4497.300000


In [8]:
df.isnull().sum()

business_id                                0
due_date                                   0
median_running_balance_2nd_m               0
median_amount_debited_2m                   0
distinct_debit_txns_1m                     0
ratio_debit_credit_2m                      0
ratio_ach_credit_freq_total_credit_1m      0
ratio_amt_drawn_mom                      125
ratio_ach_debit_credit_freq_1m             0
ratio_amt_cashin_mom                     125
ratio_revenue_1m_2m                       76
distinct_cashins_1m_prev                 125
dtype: int64

In [9]:
df0 = sc.preprocess(df,sc.features,data_params,transformer)

In [10]:
df0 = sc.model_scoring_redec(df0,sc.features,model)

In [12]:
df0.head()

,business_id,due_date,median_running_balance_2nd_m,median_amount_debited_2m,distinct_debit_txns_1m,ratio_debit_credit_2m,ratio_ach_credit_freq_total_credit_1m,ratio_amt_drawn_mom,ratio_ach_debit_credit_freq_1m,ratio_amt_cashin_mom,ratio_revenue_1m_2m,distinct_cashins_1m_prev,redec_proba,redec_score,redec_bin
0,e48c2153-6bec-4711-92b6-04f1fcbf8b62,2022-12-23,-0.551024,-0.505696,-0.280372,0.457713,0.805573,-0.275490,-0.732551,-0.350400,-0.155074,-1.054347,0.568455,431.545022,2
1,4a95d0ac-791c-4c26-ba04-27529cf78788,2022-12-24,-0.563283,-0.554756,-0.498307,-1.195758,-1.656611,-0.275416,-0.732551,-0.347744,-0.156665,2.881640,0.579811,420.188937,2
2,4a95d0ac-791c-4c26-ba04-27529cf78788,2022-12-29,-0.559013,-0.554756,-0.537224,-1.195758,-1.726065,-0.275416,-0.732551,-0.347744,-0.156665,2.881640,0.578490,421.509705,2
3,c2feb1d5-f5a9-4fde-b74a-09775639b486,2022-12-31,-0.547388,0.130604,-0.303722,2.933831,0.642969,-0.275490,-0.393512,-0.350400,-0.123138,-1.054347,0.612093,387.906989,1
4,0171a1ff-d47d-4029-b4b2-c1d2b579b857,2022-12-31,0.471843,0.267970,-0.280372,-0.579129,0.397211,-0.275490,-0.073307,0.177338,-0.150446,-0.529549,0.215790,784.210014,4


In [13]:
# df0.to_pickle('/Users/shashankgupta/Documents/code/git_project/re_decisioning/data/scored_dataset.pkl')

In [17]:
# from sqlalchemy.types import NVARCHAR
# # from conf.config import SQLQuery
# q = SQLQuery('snowflake')

# df0.to_sql(name='redec_scored_dataset',
#                  con=q.engine, 
#                  schema='prod_db.adhoc',
#                  if_exists='append', 
#                  index=False, 
#                  chunksize=16000, 
#                  method='multi',
#                  dtype={col_name: NVARCHAR for col_name in df0})

In [14]:
df0['redec_bin'].value_counts()

2    1455
3    1309
5    1264
4     898
1     799
Name: redec_bin, dtype: int64

In [15]:
len(df0['business_id'].unique())

2282

In [16]:
df0.shape

(5725, 15)